In [316]:
words = open('../data/names.txt', 'r').read().splitlines()
words[:8]


# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [317]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [318]:

n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
block_size = 3 # context length: how many characters do we take to predict the next one? 
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

def build_dataset(words):
    X, Y = [], []

    for w in words: 
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:]) 

torch.Size([182778, 3]) torch.Size([182778])
torch.Size([22633, 3]) torch.Size([22633])
torch.Size([22735, 3]) torch.Size([22735])


In [319]:
# utility functions for comparisons of manual and PyTorch gradients

def cmp(s, dt, dt_pytoch):
    if dt_pytoch is None:
        print(f'{s:15s} | Gradient is None')
        return
    ex = torch.equal(dt, dt_pytoch)
    app = torch.allclose(dt, dt_pytoch)
    maxdiff = (dt - dt_pytoch).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')


In [320]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)

# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                         generator=g) * 0.01
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),           generator=g) * 0.01
b2 = torch.randn(vocab_size,                      generator=g) * 0.01

# BatchNorm parameters
bngain = torch.ones((1, n_hidden)) * 0.01 + 1
bnbias = torch.zeros((1, n_hidden)) * 0.01
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))



In [321]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,))
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
W2.requires_grad = True
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True
  p.grad = None


4137


In [322]:
# TRAINING LOOP
# Forward pass, "chunkated" into smaller steps that are possible to backward ine at a time
# embed the characters into vectors
emb = C[Xb] # emb.shape is (batch_size, block_size, n_embd)
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hprebn.shape is (batch_size, n_hidden)
# BatchNorm layer 1
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv

hpreact = bngain * bnraw + bnbias
# Non-linearity Layer 1
h = torch.tanh(hpreact) # h.shape is (batch_size, n_hidden)
# Linear layer 2
logits = h @ W2 + b2 # logits.shape is (batch_size, vocab_size)
# cross-entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()


# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.


for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw, bnmeani,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, embcat, emb]:
    t.requires_grad_(True)
    t.retain_grad()
loss.backward()


In [334]:
# Backward pass, manually computed
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / batch_size
dprobs = (1.0 / probs) * dlogprobs
dcounts_sum_inv =  (dprobs * counts ).sum(1, keepdim=True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = -1 * counts_sum ** (-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
dnorm_logits = norm_logits.exp() * dcounts
dlogits = 1 * dnorm_logits     # not a final derivetive of logits
dlogit_maxes = -1 * (dnorm_logits).sum(1, keepdim=True)
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T
dW2 = h.T @ dlogits 
db2 = dlogits.sum(0)

dhpreact = (1 - h**2) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdims=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = -0.5 * (bnvar + 1e-5)**(-1.5) * dbnvar_inv
dbndiff = bnvar_inv * dbnraw
dbndiff2 = 1.0 / (n-1) * torch.ones_like(bndiff2) * dbnvar
dbndiff += 2.0 * bndiff * dbndiff2


dbnmeani = -( dbndiff ).sum(0)
dhprebn = 1.0 * dbndiff
dhprebn +=  torch.ones_like(hprebn)  * 1.0/n * dbnmeani

# 32, 30 |  32,64   | 64, 30
dembcat = dhprebn @  W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k, j]



# Compare manually calculated gradients with PyTorch gradients
cmp('logprobs', dlogprobs, logprobs.grad)
cmp('dprobs', dprobs, probs.grad)
cmp('dcounts_sum', dcounts_sum, counts_sum.grad)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv.grad)
cmp('dcounts', dcounts, counts.grad)
cmp('dnorm_logits', dnorm_logits, norm_logits.grad)

cmp('dlogits', dlogits, logits.grad)
cmp('dlogit_maxes', dlogit_maxes, logit_maxes.grad)

cmp('dh', dh, h.grad)
cmp('dW2', dW2, W2.grad)
cmp('db2', db2, b2.grad)
cmp('dhpreact', dhpreact, hpreact.grad)
cmp('dbngain', dbngain, bngain.grad)
cmp('dbnraw', dbnraw, bnraw.grad)
cmp('dbnbias', dbnbias, bnbias.grad)

cmp('dbnvar_inv', dbnvar_inv, bnvar_inv.grad)
cmp( 'dbnvar', dbnvar, bnvar.grad)
cmp('dbndiff', dbndiff, bndiff.grad)
cmp('dbndiff2', dbndiff2, bndiff2.grad)

cmp('dhprebn', dhprebn, hprebn.grad)
cmp('dbnmeani', dbnmeani, bnmeani.grad)

cmp('dembcat', dembcat, embcat.grad)
cmp('dW1', dW1, W1.grad)
cmp('db1', db1, b1.grad)

cmp('demb', demb, emb.grad)




# plt.imshow((F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes).detach().numpy())


logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
dprobs          | exact: True  | approximate: True  | maxdiff: 0.0
dcounts_sum     | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
dcounts         | exact: True  | approximate: True  | maxdiff: 0.0
dnorm_logits    | exact: True  | approximate: True  | maxdiff: 0.0
dlogits         | exact: True  | approximate: True  | maxdiff: 0.0
dlogit_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
dh              | exact: True  | approximate: True  | maxdiff: 0.0
dW2             | exact: True  | approximate: True  | maxdiff: 0.0
db2             | exact: True  | approximate: True  | maxdiff: 0.0
dhpreact        | exact: True  | approximate: True  | maxdiff: 0.0
dbngain         | exact: True  | approximate: True  | maxdiff: 0.0
dbnraw          | exact: True  | approximate: True  | maxdiff: 0.0
dbnbias         | exact: True  | approximate: True  | maxdiff:

In [324]:
try:
    torch.autograd.grad(loss, W2, retain_graph=True)
    print("Loss depends on W2")
except RuntimeError:
    print("Loss does not depend on W2")

Loss does not depend on W2


In [325]:
dlogits.shape, h.shape, W2.shape, b2.shape
b2.grad

tensor([-0.1190, -0.0571,  0.0059,  0.0369,  0.0368, -0.1187,  0.0377,  0.0376,
        -0.0253,  0.0367,  0.0369,  0.0053, -0.0252,  0.0369, -0.0574,  0.0373,
         0.0370,  0.0366, -0.0258, -0.0561,  0.0375,  0.0366,  0.0061,  0.0369,
         0.0373, -0.0570,  0.0055])

In [336]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.292738437652588 diff: 0.0


In [337]:
# backward pass
# dloss/d_i_logits = ???
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('dlogits', dlogits, logits.grad)

dlogits         | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09


In [339]:
plt.figure(figsize=(8, 8))
plt.imshow(dlogits.detach().numpy(), cmap=plt.cm.gray)

AttributeError: Figure.set() got an unexpected keyword argument 'figuresize'